In [ ]:

import tweepy
import json

### Get these from Twitter dev account
CONSUMER_KEY = ''
CONSUMER_SECRET = ''
ACCESS_TOKEN = ''
ACCESS_TOKEN_SECRET = ''


# Loading keys etc.
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

s = tweepy.Stream(auth, 1, tweet_mode='extended')


auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)


## function  to save data to SQLite
import sqlite3

def create_table(user,userdesc,numfollowers,text,retweet_cnt,created_at,twitter_id,geoloc,location,name):  # creates table
    ### Sql lite likes strings, had errors before conversion
    
    twitter_id = str(twitter_id)
    user=str(user)
    userdesc = str(userdesc)
    text = str(text)
    
    numfollowers = str(numfollowers)
    retweet_cnt = str(retweet_cnt)
    created_at = str(created_at)
    
    geoloc = str(geoloc)
    location = str(location)
    name = str(name)
       

    print(text)
    
    ## creating the database and table if it does not exsist, then loading the data
    
    conn = sqlite3.connect('covid_twitter2.db', timeout = 30)
    c = conn.cursor()
    c.execute('CREATE TABLE IF NOT EXISTS PA(twitter_id TEXT, user TEXT, userdesc TEXT, numfollowers TEXT,text TEXT, retweet_cnt TEXT, created_at TEXT,geoloc TEXT,location TEXT,name Text)')
    c.execute('INSERT INTO PA(twitter_id,user,userdesc,numfollowers,text,retweet_cnt,created_at,geoloc,location,name)VALUES(?,?,?,?,?,?,?,?,?,?)',(twitter_id, user, userdesc, numfollowers,text,retweet_cnt,created_at,geoloc,location,name))                    
    conn.commit()


### Main
class MyStreamListener(tweepy.StreamListener):
    def on_status(self, status):
        try:
            text = status.extended_tweet["full_text"]
            print(text)
        except AttributeError:
            text = status.text

        user = status.user.screen_name
        userdesc = status.user.description
        numfollowers = status.user.followers_count
        twitter_id = status.id
        geoloc = status.geo
        name = status.user.name
        location = status.user.location
       
        retweet_cnt = status.retweet_count
        created_at = status.created_at
        create_table(user,userdesc,numfollowers,text,retweet_cnt,created_at,twitter_id,geoloc,location,name) 
            
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener=myStreamListener , tweet_mode='extended')

 
myStream.filter(track=['Covid-19 Pennsylvania'])

